In [106]:
import json 
import re
from bs4 import BeautifulSoup
import requests

state_dict = {"ca":"California","sc":"South Carolina"} #Set up dictionary that matches state initials to state name
sport_set = {"basketball","baseball","football"} #Set up sports set 
redundant_words = {"maxpreps","http","https","www","com"}
relevant_words = {}
with open("Resources/SiteInfo.json", "r") as f:
    json_dev_file = json.load(f)

def get_html(URL): #gets html in text
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    page = requests.get(URL,headers=header) 
    soup = BeautifulSoup(page.text,'html.parser') #take html
    return soup

##going with simplicity here, 
def grab_hrefs(page_elem):
    tags = []
    tags = page_elem.find_all('a')
    links = [tag['href'] for tag in tags if 'href' in tag.attrs] #find links
    return links

def grab_id_parent(page_elem,pg_id):
    return page_elem.find(id=pg_id)
def grab_scripts(page_elem,regex = re.compile('')):
    #return [script for script in page_elem.find_all("script") if regex.search(script.attrs['src'])]
    return page_elem.find_all("script")
    
#IN USE NOW 
def grab_team_hrefs(page_elem):
    links = grab_hrefs(page_elem)

    ####REWORK vvvvv to work with other states via an container
    team_links = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"]
    ####HARDCODED RN but can use relevant words from parse_url_context in the future to decipher relevant links
    return team_links

def grab_player_hrefs(page_elem): #eventually should make respective classes that override this function
    links = grab_hrefs(page_elem)
    relevant_pattern = r'careerid=[a-z0-9]+' #eventually, it should read off json to get a regex depending on site
    player_links = [link for link in links if re.search(relevant_pattern,link)]
    return player_links
#WIP

def grab_player_info(page_elem):
    ##there is a big script.
    player_scripts = grab_id_parent(ex_player_pg, '__NEXT_DATA__')
    return
#WIP might not need
def get_team_URL(page_html):
    #try to guess which link is the teams URL on a given site
    return

In [2]:
laf_ball_roster_html = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/basketball/roster/")

In [3]:
team_page_html = get_html("https://www.maxpreps.com/ca/basketball/schools/") 

In [4]:
##IF WANT TO TEST TEAMS, THERE ARE 200 TEAMS 

In [21]:
team_links = grab_team_hrefs(team_page_html)
print(len(team_links))

200


In [25]:
#FOR SOME REASON THEY HAVE A SPACE which throws off search.
#develop context based title finding 
member_links = grab_roster_hrefs(laf_ball_roster_html)
print(f'{len(member_links)} players\n{member_links}') #as of rn, there are 13 players on this team


13 players
['/ca/lafayette/acalanes-dons/athletes/preston-hilsabeck/?careerid=hpb7k1kir1dm6', '/ca/lafayette/acalanes-dons/athletes/cameron-hood/?careerid=dsg8ao87v6ep9', '/ca/lafayette/acalanes-dons/athletes/julian-hood/?careerid=56mf96hmdg7o8', '/ca/lafayette/acalanes-dons/athletes/justice-hembrador/?careerid=2kfkn0i2o19bb', '/ca/lafayette/acalanes-dons/athletes/sam-phillips/?careerid=qd4gfkvejm577', '/ca/lafayette/acalanes-dons/athletes/bryce-mansour/?careerid=hblf6vc03u0bb', '/ca/lafayette/acalanes-dons/athletes/aj-hastings/?careerid=5s34l10a9ftee', '/ca/lafayette/acalanes-dons/athletes/evan-palmer/?careerid=e40dlmenhoaaa', '/ca/lafayette/acalanes-dons/athletes/shea-stahl/?careerid=d8rohq05aqrvf', '/ca/lafayette/acalanes-dons/athletes/jon-macleod/?careerid=23f38c690m8td', '/ca/lafayette/acalanes-dons/athletes/gavin-dodge/?careerid=2eorqhnhgcb44', '/ca/lafayette/acalanes-dons/athletes/aakash-agarwal/?careerid=9u5jbbjt91tm6', '/ca/lafayette/acalanes-dons/athletes/landon-deily/?career

In [76]:
####PLAYER LOGIC FROM ROSTER#####

#given a team, find the roster link <-- skip since we hardcode it rn

#TODO: from rosters page, find player links
#it is also just maxpreps.com{whatever player we found from roster} ... same goes for any links we find; it is just maxpreps.com{found links}
site_url = json_dev_file['maxpreps']['site_url']
ex_player_pg = get_html(f"{site_url}{member_links[-1]}")

In [114]:
##CAN USE FIRESTORE ATTRIBUTES FOR BASKETBALL TO LOOK FOR HEADERS THAT HAVE THAT INFO
player_json_script = grab_id_parent(ex_player_pg, '__NEXT_DATA__')
json_data = json.loads(player_json_script.string)
with open('ex_player_info.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)

In [9]:
#TODO tentative
    #continue on roster directory logic -- DONE
    #continue on player page logic -- DONE 
    #make it create json file that maps teams(from DB) to links
        #"data_pipeline_shortcuts":{
        #"team_url_pattern": "",
        #"team_page_url_pattern": "",
        #"roster_page_url_pattern": "",
        #"player_page_url_pattern": ""
        #},
    #make known path for specific website and add to relevant words
    #eventually make datastructures for these things: teams and players so we can serialize and deserialize from db for QoL

In [10]:
##EVERYTHING BELOW IS JUST WIP 

In [11]:
redundant_words = {"maxpreps","http","https","www","com"} #could also be relevant words when looking for links that match page context 
URL = "https://www.maxpreps.com/ca/basketball/schools/"
url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
url_words

['ca', 'basketball', 'schools']

In [ ]:
target_text = soup.find_all(['h1','h2','h3'])
target_text[-1].get_text()
target_text #WIP on pinpointing exact div-- for the far future

In [ ]:
def parse_url_context(URL, redundant_words): # remove maxpreps and https:// etc but keep keywords like ca
    # sc, Basketball, Boys to use and find relevant themes. This way, we can remove redundant words from links within page that may point to itself
    #can use these words and additional context to generate likelihood of teams div
    #not use right now
    #chop up url
    url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
    return url_words
